In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from pathlib import Path

2023-12-05 10:39:03.015352: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
DATA_PATH = Path("/scratch/ajb5d/ecg/tfrecords/")
TRAIN_RECS = list(DATA_PATH.glob("train*.tfrecords"))
VAL_RECS = list(DATA_PATH.glob("val*.tfrecords"))

BATCH_SIZE = 64

record_format = {
    'ecg/data': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'age': tf.io.FixedLenFeature([], tf.float32),
    'gender': tf.io.FixedLenFeature([], tf.int64),
    'hospital_expire_flag': tf.io.FixedLenFeature([], tf.float32),
    'icu_expire_flag': tf.io.FixedLenFeature([], tf.float32),
}

def _parse_record(record):
    example = tf.io.parse_single_example(record, record_format)
    ecg_data = tf.reshape(example['ecg/data'], [5000,12])
    label = example['icu_expire_flag']
    return (ecg_data, example['age'], example['gender']), label

@tf.function
def drop_na_labels(x,y):
    return not tf.math.reduce_any(tf.math.is_nan(y))

@tf.function
def drop_na_age(x,y):
    return not tf.math.reduce_any(tf.math.is_nan(x[1]))

def load_dataset(filenames):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(_parse_record, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.filter(drop_na_labels)
    dataset = dataset.filter(drop_na_age)
    return dataset

def get_dataset(filenames, labeled=True):
    dataset = load_dataset(filenames)
    dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

train_dataset = get_dataset(TRAIN_RECS)
val_dataset = get_dataset(VAL_RECS)

In [12]:
model = keras.models.load_model("data/models/resnet-age.keras")

In [13]:
for i in range(len(model.layers)):
    model.layers[i].trainable = False

In [14]:
model.layers[-2]

In [15]:
x = model.layers[-2].output
age_input = tf.keras.layers.Input(shape=(1,), name="age_input")
gender_input = tf.keras.layers.Input(shape=(1,), name="gender_input")
x = keras.layers.Add(name="merge")([x, age_input, gender_input])
x = tf.keras.layers.Dense(512, name="tl_dense_3")(x)
x = tf.keras.layers.Dense(1, activation='sigmoid', name = "tl_dense_4")(x)

new_model = keras.Model([model.input, age_input, gender_input],outputs=x)

new_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy', tf.keras.metrics.AUC()]
)

In [16]:
from datetime import datetime
import os

def make_checkpoint_dir(data_path, label):
    current_datetime = datetime.now()
    formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")
    output_dir = f"{label}-{formatted_datetime}"
    output_path = f"{data_path}/{output_dir}"
    
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    return output_path

model_name = "tl-age-mort"
output_path = make_checkpoint_dir("data/models", model_name)

print(f"Model: {model_name} Run Path: {output_path}")

Model: tl-age-mort Run Path: data/models/tl-age-mort-2023-12-05_11-01-07


In [17]:
callbacks = [
    tf.keras.callbacks.TerminateOnNaN(),
    tf.keras.callbacks.ReduceLROnPlateau(),
    tf.keras.callbacks.ModelCheckpoint(output_path, save_best=True),
    tf.keras.callbacks.CSVLogger(f"data/models/{model_name}-history.csv")
]

history = new_model.fit(train_dataset, epochs=10, validation_data=val_dataset, callbacks=callbacks)

Epoch 1/10
    453/Unknown - 56s 109ms/step - loss: 135.5873 - accuracy: 0.7430 - auc_1: 0.4978INFO:tensorflow:Assets written to: data/models/tl-age-mort-2023-12-05_11-01-07/assets


INFO:tensorflow:Assets written to: data/models/tl-age-mort-2023-12-05_11-01-07/assets


453/453 [==============================] - 80s 162ms/step - loss: 135.5873 - accuracy: 0.7430 - auc_1: 0.4978 - val_loss: 0.4474 - val_accuracy: 0.8493 - val_auc_1: 0.4757 - lr: 0.0010
Epoch 2/10
453/453 [==============================] - ETA: 0s - loss: 0.5127 - accuracy: 0.8314 - auc_1: 0.4966INFO:tensorflow:Assets written to: data/models/tl-age-mort-2023-12-05_11-01-07/assets


INFO:tensorflow:Assets written to: data/models/tl-age-mort-2023-12-05_11-01-07/assets


453/453 [==============================] - 76s 161ms/step - loss: 0.5127 - accuracy: 0.8314 - auc_1: 0.4966 - val_loss: 0.4384 - val_accuracy: 0.8493 - val_auc_1: 0.4750 - lr: 0.0010
Epoch 3/10
453/453 [==============================] - ETA: 0s - loss: 0.4713 - accuracy: 0.8404 - auc_1: 0.4902INFO:tensorflow:Assets written to: data/models/tl-age-mort-2023-12-05_11-01-07/assets


...

453/453 [==============================] - 137s 282ms/step - loss: 0.5531 - accuracy: 0.8257 - auc_1: 0.4869 - val_loss: 0.4347 - val_accuracy: 0.8493 - val_auc_1: 0.4760 - lr: 0.0010
Epoch 9/10
453/453 [==============================] - ETA: 0s - loss: 0.4742 - accuracy: 0.8404 - auc_1: 0.4874INFO:tensorflow:Assets written to: data/models/tl-age-mort-2023-12-05_11-01-07/assets


INFO:tensorflow:Assets written to: data/models/tl-age-mort-2023-12-05_11-01-07/assets


453/453 [==============================] - 88s 184ms/step - loss: 0.4742 - accuracy: 0.8404 - auc_1: 0.4874 - val_loss: 0.4475 - val_accuracy: 0.8493 - val_auc_1: 0.4755 - lr: 0.0010
Epoch 10/10
453/453 [==============================] - ETA: 0s - loss: 0.4648 - accuracy: 0.8404 - auc_1: 0.4899INFO:tensorflow:Assets written to: data/models/tl-age-mort-2023-12-05_11-01-07/assets


INFO:tensorflow:Assets written to: data/models/tl-age-mort-2023-12-05_11-01-07/assets


453/453 [==============================] - 82s 171ms/step - loss: 0.4648 - accuracy: 0.8404 - auc_1: 0.4899 - val_loss: 0.4399 - val_accuracy: 0.8493 - val_auc_1: 0.4755 - lr: 0.0010
